## book rentalshop on Jupyter NoteBook

#### 임포트 해주기
> import cx_Oracle as ora

In [1]:
import cx_Oracle as ora

## 함수 만들기
#### DB연결 함수
호출방법
> myconn()
> 또는 myconn = myconn()

In [4]:
def myconn():
    dsn = ora.makedsn('localhost', 1521, service_name='orcl')
    conn = ora.connect(user='scott', password='tiger', dsn=dsn, encoding='utf-8')
    return conn

#### divtbl 전체 조회 함수(SELECT)
호출방법
> getAllDataFromDivtbl(conn) # conn은 연결 객체

In [5]:
# 1. divtbl에서 데이터 조회 : SELECT
def getAllDataFromDivtbl(conn):
    cur = conn.cursor()
    query = 'SELECT * FROM divtbl'
    for row in cur.execute(query):
        print(row)

#### divtbl에 신규 장르 정보 입력(INSERT)
호출방법

tuple = (division, names)
> setDataIntoDivtbl(conn, tuple)

In [6]:
# 2. divtbl에 새로운 데이터 입력 : INSERT
def setDataIntoDivtbl(conn, tup):
    cur = conn.cursor()
    query = '''INSERT INTO divtbl (division, names) 
                VALUES (:1, :2) '''
    cur.execute(query, tup)
    cur.close()
    conn.commit()

#### membertbl에서 회원 정보 조회(SELECT)
호출방법
> getSomeDataFromMembertbl(conn)

In [16]:
# 3. membertbl에서 데이터 조회 : SELECT
def getSomeDataFromMembertbl(conn):
    cur = conn.cursor()
    query = '''SELECT names, levels, addr, mobile, email 
                 FROM membertbl
                WHERE addr LIKE '서울%'
                  AND UPPER(email) LIKE '%@NAVER.COM'
                ORDER BY idx DESC'''
    for row in cur.execute(query):
        print(row)
    cur.close()

#### membertbl에 새로운 회원 정보 입력(INSERT)
호출방법
> setNewMemberIntoMembertbl(conn, tuple)


In [19]:
# 4. membertbl에 새 데이터 입력 : INSERT
def setNewMemberIntoMembertbl(conn, tup):
    cur = conn.cursor()
    query = '''SELECT ROWNUM, idx
                FROM (
                    SELECT idx FROM membertbl
                    ORDER BY idx DESC  
                        ) 
                WHERE ROWNUM = 1'''
    cur.execute(query)
    idx = cur.fetchone() # 해당 테이블에 데이터가 하나도 없을 경우에는 예외가 발생할 수도 있다
    if idx is None: # 데이터가 없을 경우 (None)
        idx = 0 # idx에 0 넣어주기
    else: # None이 아니라면
        idx = idx[1] # 두번째 idx에 두번째 값인 인덱스 값을 넣어주기 

    intup = (idx + 1, tup[0], tup[1], tup[2], tup[3])

    query = '''INSERT INTO membertbl 
                        (idx, names, levels, userid, password)
                VALUES (:1, :2, :3, :4, :5)'''
    cur.execute(query, intup)
    cur.close()
    conn.commit()

#### membertbl의 데이터 수정(UPDATE)
호출방법
> setChangeMemberFromMembertbl(conn, tuple)

In [22]:
# 5. membertbl 데이터를 수정 : UPDATE
def setChangeMemberFromMembertbl(conn, tup):
    cur = conn.cursor()
    query = '''UPDATE membertbl
                SET addr = :1
                , mobile = :2
                , email = :3
                WHERE idx = :4'''
    cur.execute(query, tup)
    conn.commit()

#### divtbl에 임의 데이터 삭제(DELETE)
호출방법
> deleteDivision(conn, division)

In [26]:
def deleteDivision(conn, division):
    cur = conn.cursor()
    query = 'DELETE FROM divtbl WHERE division = :1'
    cur.execute(query, (division,))
    conn.commit()

#### 책 대여 프로그램 시작

In [7]:
scott_conn = myconn()
print('접속')

접속


#### 책 장르 정보 조회
> getAllDataFromDivtbl(scott_conn)

In [18]:
    print('--책 장르 정보 조회--')
    getAllDataFromDivtbl(scott_conn)

--책 장르 정보 조회--
('B001', '공포/스릴러')
('B002', '로맨스')
('B003', '무협')
('B004', '전쟁/역사')
('B005', '추리')
('B006', 'SF/판타지')
('I001', '프로그래밍')
('I003', '네트워크')
('I005', '정보보안')
('I101', '이름 몰라')
('I102', '이름 몰라')


#### 신규 장르 정보 입력
> setDataIntoDivtbl(scott_conn, tup)

In [13]:
    print('--책 장르 정보 추가--')
    division = 'I102'
    names = '이름 몰라'
    tup = (division, names)
    setDataIntoDivtbl(scott_conn, tup)
    getAllDataFromDivtbl(scott_conn)
    print('--정보를 입력했습니다--')

--책 장르 정보 추가--
('B001', '공포/스릴러')
('B002', '로맨스')
('B003', '무협')
('B004', '전쟁/역사')
('B005', '추리')
('B006', 'SF/판타지')
('I001', '프로그래밍')
('I003', '네트워크')
('I005', '정보보안')
('I101', '이름 몰라')
('I102', '이름 몰라')
--정보를 입력했습니다--


#### 회원 정보 조회
> getSomeDataFromMembertbl(scott_conn)

In [17]:
    print('--회원의 정보를 조회--')
    getSomeDataFromMembertbl(scott_conn)

--회원의 정보를 조회--
('김수연', 'A', '서울시 강서구', '010-7625-0676', 'KSY@NAVER.COM')
('김얼', 'A', '서울시 종로구', '010-5341-4736', 'KE91@NAVER.COM')
('홍건의', 'D', '서울시 성동구', '010-9341-0128', 'HGE@NAVER.COM')
('박상섭', 'C', '서울시 노원구', '010-5341-0128', 'PSS@NAVER.COM')
('이규형', 'A', '서울시 금천구', '010-2967-1016', 'LGH@NAVER.COM')


#### 신규 회원 정보 입력
> setNewMemberIntoMembertbl(scott_conn, tup)

In [21]:
    print('--신규 회원 입력--')
    names = 'go'
    levels = 'A'
    userid = 'gogogo'
    password = 1541
    tup = (names, levels, userid, password)
    setNewMemberIntoMembertbl(scott_conn, tup)
    print('--신규 회원 저장 성공--')

--신규 회원 입력--
--신규 회원 저장 성공--


#### 회원정보 수정
> deleteDivision(conn, division)

In [25]:
    print('--회원 정보 수정--')
    idx = 28
    addr = '뉴욕시티'
    mobile = 000-000-000
    email = 'new_york_jo_a'
    tup = (addr, mobile, email, idx) # idx가 마지막인 이유는 마지막에 쓰는 WHERE절에 가장 처음 입력한 idx가 들어가기 때문임
    setChangeMemberFromMembertbl(scott_conn, tup)
    print('--기존회원 정보 수정 완료--')

--회원 정보 수정--
--기존회원 정보 수정 완료--


#### 책 장르 정보 삭제
> deleteDivision(scott_conn, division)

In [28]:
    print('--책 장르 정보 삭제--')
    division = 'I102'
    deleteDivision(scott_conn, division)
    print('--삭제 성공--')

--책 장르 정보 삭제--
--삭제 성공--
